# Acartia.io map vis
    
## Purpose
A Jupyter notebook for visualizing the open observations available at arcartia.io. Simple (brain dead) color visualization of time on points. Eventually, this could turn into a modeling tool for filtering observations, doing simple velocity modeling, etc.

## Author
Christian P Sarason (cpsarason.github.io)

## Copyright
CC-BY (https://creativecommons.org/licenses/by/2.0/) for Orcasound.net 2022

## Requirements

Most easily used either by running locally via Jupyter (

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt
import datetime
!pip install --upgrade -q pyproj

     |████████████████████████████████| 6.3 MB 7.2 MB/s 


# Read in date from Acartia.io into JSON object and create dataframe


In [ ]:
import requests
response = requests.get('https://acartia.io/api/v1/sightings/current')

#print(response.text)

acartia_response = json.loads(response.text)
acartia_response
acartia_df = pd.DataFrame(acartia_response)
acartia_df.longitude = acartia_df.longitude.astype('f')
acartia_df.latitude = acartia_df.latitude.astype('f')
acartia_df['obs_date'] = pd.to_datetime(acartia_df['created'])
acartia_df['elapsed_timedelta'] = acartia_df['obs_date'] - pd.datetime.now()
acartia_df['elapsed_seconds'] = acartia_df['elapsed_timedelta'] / np.timedelta64(1, 's')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if sys.path[0] == '':


# Plot up all science profile data

In [ ]:
acartia_df


,ssemmi_id,data_source_name,data_source_entity,data_source_id,created,photo_url,no_sighted,latitude,longitude,type,...,trusted,data_source_comments,profile,entry_id,ssemmi_date_added,submitter_did,signature,obs_date,elapsed_timedelta,elapsed_seconds
0,SPOTTER 149524,Spotter-API,Conserve.io,149524,2022-09-10 17:30:00,,6,48.201279,-122.744278,Killer Whale (Orca),...,1,[Orca Network] 6 Biggs traveling shoreline nor...,{'name': 'spotter'},ef357c65-a147-4ecf-ae04-12de9ad23b8d,Sun Sep 11 2022 18:34:08 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': 'fe39f41fad1185167d6cc1c2c4a050d6e848203...,2022-09-10 17:30:00,-4 days +15:04:11.073735,-291348.926265
1,SPOTTER 149523,Spotter-API,Conserve.io,149523,2022-09-11 17:14:00,,6,47.994801,-122.268784,Killer Whale (Orca),...,1,"[HIWW] reported as 5, but video reviewed by Or...",{'name': 'spotter'},e674132d-f7ab-44c8-b3ed-dc96c27421e7,Sun Sep 11 2022 18:27:28 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': '6d1f1a8bebc505fe25912c44ab2b87690e22f20...,2022-09-11 17:14:00,-3 days +14:48:11.073735,-205908.926265
2,SPOTTER 149507,Spotter-API,Conserve.io,149507,2022-09-11 00:20:00,,5,44.689930,-124.065887,Grey,...,1,,{'name': 'spotter'},d5ad2cc4-5133-485a-84e9-e8c510e5e9aa,Sun Sep 11 2022 18:10:05 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': 'e4f199b41b0bdc1f7eeeff2aed7f720aa65f0f7...,2022-09-11 00:20:00,-4 days +21:54:11.073735,-266748.926265
3,SPOTTER 149460,Spotter-API,Conserve.io,149460,2022-09-10 00:22:28,,1,44.850952,-124.100510,Gray Whale,...,1,,{'name': 'spotter'},db81e85f-9ab3-47e3-9718-c4c8b7f2b618,Sun Sep 11 2022 18:10:02 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': '820eba73ea5751b3c5b69bf623f30ab1a970531...,2022-09-10 00:22:28,-5 days +21:56:39.073735,-353000.926265
4,SPOTTER 149488,Spotter-API,Conserve.io,149488,2022-09-10 22:32:11,,5,47.137459,-122.916656,Harbor Porpoise,...,1,,{'name': 'spotter'},79c02b6e-9f0e-4319-bf85-522497fbf753,Sun Sep 11 2022 18:09:52 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': 'a6e29923e956bdf774f2398680da71506ae072b...,2022-09-10 22:32:11,-4 days +20:06:22.073735,-273217.926265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,SPOTTER 149648,Spotter-API,Conserve.io,149648,2022-09-11 18:54:00,,1,48.198978,-122.576408,Humpback,...,1,[Orca Network] Malachite milling (Bart Rulon),{'name': 'spotter'},4869323d-d129-43db-a2fe-8a92fcfe9fce,Tue Sep 13 2022 19:48:09 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': 'e2c73cb16db287f39c5add77d2aff87737e6d71...,2022-09-11 18:54:00,-3 days +16:28:11.073735,-199908.926265
273,SPOTTER 149649,Spotter-API,Conserve.io,149649,2022-09-11 20:17:00,,1,48.177689,-122.554779,Humpback,...,1,[Orca Network] Southbound. Presume Malachite....,{'name': 'spotter'},e82e5c3b-cb52-4945-8fb1-b67fb168ee13,Tue Sep 13 2022 19:50:24 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': '6f5e27edd4a95e4745dbeb81c1f5a0eca8cf90f...,2022-09-11 20:17:00,-3 days +17:51:11.073735,-194928.926265
274,SPOTTER 149650,Spotter-API,Conserve.io,149650,2022-09-11 22:00:00,,1,48.192032,-122.559334,Humpback,...,1,[Orca Network] Northwest travel. Presume Mala...,{'name': 'spotter'},4f2b5ce6-5354-4276-b821-5ca4c0bf55c0,Tue Sep 13 2022 19:53:00 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': '13b883808bb4252ac23a828f3538d081faf5722...,2022-09-11 22:00:00,-3 days +19:34:11.073735,-188748.926265
275,SPOTTER 149651,Spotter-API,Conserve.io,149651,2022-09-11 22:20:00,,1,47.308121,-122.512810,Minke Whale,...,1,[Orca Network] direction unknown. (Lindsay Mum...,{'name': 'spotter'},687a6e8d-600e-43a9-be64-aa541c9e531c,Tue Sep 13 2022 19:55:57 GMT+0000 (Coordinated...,did:ethr:0x20fd1096eaafb242a88272e20d7a77b552f...,{'r': 'a54bb3652dca8acac18f36f709c3c5dea5b6637...,2022-09-11

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource

from bokeh.models.tools import HoverTool
from bokeh.models import LinearAxis, Range1d, CategoricalColorMapper
import bokeh.palettes as bok_pal

from bokeh.models import ColumnDataSource, ColorBar
from bokeh.palettes import Spectral6, viridis, d3,Turbo256
from bokeh.transform import linear_cmap
from bokeh.palettes import brewer

from bokeh.io import output_notebook
output_notebook()




output_file('acartia_map.html')
acartiaDataSource = ColumnDataSource(acartia_df)


tools="box_zoom,wheel_zoom,pan,hover,box_select,lasso_select,reset"
scienceFormatters={'float': 'printf',
            'dive' : 'printf', 
            'pressure': 'printf',
            'temperature': 'printf',
            'salinity': 'printf',
            'density': 'printf',
           }

# colors from https://malouche.github.io/notebooks/scatter_bokeh.html
# mapper = linear_cmap(field_name='dive', 
#                      palette=Spectral6 ,
#                      low=min(science_df['dive']) ,
#                      high=max(science_df['dive']))

#num_dives = 21


colormap = {8: '#440154',
 9: '#46317E',
 10: '#365A8C',
 11: '#277E8E',
 12: '#1EA087',
 13: '#49C16D',
 14: '#9DD93A',
 15: '#FDE724'}

p1_dotsize = 5
rainbow_time_map = linear_cmap(field_name='elapsed_seconds', palette=Turbo256, low=min(acartia_df['elapsed_seconds']), high=max(acartia_df['elapsed_seconds']))
# ### Color by dive stages
# ## add dive stage coloring
# use whatever palette you want...

  

##################### map section
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors, ESRI_IMAGERY,WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
from bokeh.io import output_notebook, show
from bokeh.models import WMTSTileSource

import warnings

warnings.filterwarnings("ignore")
#pd.set_option("max_columns", 30)

from pyproj import Proj, transform

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')


#world_lon1, world_lat1 = transform(outProj,inProj,-180,-85)
#world_lon2, world_lat2 = transform(outProj,inProj,180,85)

llx, lly = transform(outProj, 
                     inProj, 
                     acartia_df['longitude'].min() - 1,
                     acartia_df['latitude'].min() - 1)

urx, ury = transform(outProj,
                     inProj,
                     acartia_df['longitude'].max() + 1,
                     acartia_df['latitude'].max() + 1)


lons, lats = [], []
for lon, lat in list(zip(acartia_df['longitude'],acartia_df['latitude'])):
    x, y = transform(outProj, inProj, lon, lat)
    lons.append(x)
    lats.append(y)

acartia_df['merc_x_start'] = lons
acartia_df['merc_y_start'] = lats

#world_lon1, world_lat1 = transform(outProj,inProj,fallrise_df['merc_x_start'].min(),fallrise_df['merc_y_start'].min())
#world_lon2, world_lat2 = transform(outProj,inProj,fallrise_df['merc_x_start'].max(),fallrise_df['merc_y_start'].max())

esri_img = get_provider(ESRI_IMAGERY)
esri_img = get_provider(ESRI_IMAGERY)

esri_ocean = WMTSTileSource(
    url = 'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{z}/{y}/{x}', 
    attribution = 'test',
    max_zoom = 13)


obs_map = figure(plot_width=800, plot_height=700,
             x_range=(llx, urx),
             y_range=(lly, ury),
             x_axis_type="mercator", 
             y_axis_type="mercator",
            )

obs_map.add_tile(esri_ocean)
obs_map.circle(x='merc_x_start', y='merc_y_start',
          size=10,
          fill_color=rainbow_time_map,
          line_color=rainbow_time_map,
          fill_alpha=0.8,
          source=acartia_df)
obs_map.xgrid.grid_line_color = None
obs_map.ygrid.grid_line_color = None

#################### end map section


show(layout([
    [obs_map],
    ]))


